In [1]:
from model import *
from warnings import filterwarnings

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/scratch/pratim/miniconda3/envs/grammar/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
positive_encodings, negative_encodings = load_complete_dataset(n_files=10)

100%|██████████| 10/10 [00:02<00:00,  3.65it/s]


In [20]:
class MLPClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(0.01)
        self.sinu = nn.SiLU()
        self.batchnorm = nn.BatchNorm1d(hidden_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.batchnorm(x)
        x = self.sinu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.batchnorm(x)
        x = self.sinu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.softmax(x)
        return x

In [29]:
batch_size = 16

dataset = torch.concat([positive_encodings, negative_encodings], dim=0)
labels = torch.tensor([1] * len(positive_encodings) + [0] * len(negative_encodings))

dataset = torch.utils.data.TensorDataset(dataset, labels)

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(0.8 * len(dataset)), len(dataset) - int(0.8 * len(dataset))])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [30]:
learning_rate = 1e-5
num_epochs = 15
input_size = 3072
hidden_size = 1024
n_classes = 2

model = MLPClassifier(input_size, hidden_size, n_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for i, (data, labels) in enumerate(train_loader):

        data = data.to(device).float()
        labels = labels.to(device).long()

        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'[Epoch {epoch + 1}] | Loss: {round(loss.item(), 3)}')

[Epoch 1] | Loss: 0.524
[Epoch 2] | Loss: 0.581
[Epoch 3] | Loss: 0.538
[Epoch 4] | Loss: 0.568
[Epoch 5] | Loss: 0.45
[Epoch 6] | Loss: 0.448
[Epoch 7] | Loss: 0.403
[Epoch 8] | Loss: 0.461
[Epoch 9] | Loss: 0.375
[Epoch 10] | Loss: 0.322
[Epoch 11] | Loss: 0.356
[Epoch 12] | Loss: 0.335
[Epoch 13] | Loss: 0.422
[Epoch 14] | Loss: 0.425
[Epoch 15] | Loss: 0.321


In [31]:
train_acc = 0

with torch.no_grad():
    for data, labels in test_loader:
        data = data.to(device).float()
        labels = labels.to(device).long()

        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        train_acc += (predicted == labels).sum().item()

print(f'Test Accuracy: {round(train_acc / len(test_dataset), 3)}')

Test Accuracy: 0.871


In [32]:
test_acc = 0
with torch.no_grad():
    for data, labels in test_loader:
        data = data.to(device).float()
        labels = labels.to(device).long()

        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        test_acc += (predicted == labels).sum().item()

print(f'Test Accuracy: {round(test_acc / len(test_dataset), 3)}')

Test Accuracy: 0.874
